# load pennai ML options

In [1]:
import json                                                                          
import numpy as np                                                                   
from json import JSONDecoder                                                         
from functools import partial                                                        
                                                                                     
path = 'projects_newlines.json'                                
data,alldata=[],[]                                                                   
with open(path) as f:                                                                
    for line in f:                                                                   
        alldata.append(json.loads(line))                                             
    # for data in json_parse(f):                                                     
    #     print(data)                                                                
    #     alldata.append(data)                                                       
pennai_ml_opts = {}                                                                         
import pdb                                                                           
from sklearn.model_selection import ParameterGrid                                    
                                                                                     
original_pennai_ml_opts = {}
for d in alldata:                                                                    
    print('ml:',d['name'])                                                           
                                                                                     
    params = d['schema'].keys()                                                      
    choices = {}                                                                     
    for p in params:                               
        if 'values' in d['schema'][p]['ui']:
            choices[p] = d['schema'][p]['ui']['values']                                 
        else:
            choices[p] = d['schema'][p]['ui']['choices']                                 
        print('param:',p,', options:',choices[p])
    # add None to DecisionTreeClassifier's max_depth option.
    if d['name'] == 'DecisionTreeClassifier':
        choices['max_depth'].append(None)
#     pdb.set_trace()
    original_pennai_ml_opts[d['name']] = choices.keys()
    param_grid = list(ParameterGrid(choices))                                        
#     print('param_grid contains',len(param_grid),'options')                           
    pennai_ml_opts[d['name']] = param_grid                                                  
print('pennai_ml_opts:',len(pennai_ml_opts))      
for key in pennai_ml_opts.keys():
    print(key,'options:',len(pennai_ml_opts[key]))

ml: BernoulliNB
param: alpha , options: [0.001, 0.01, 0.1, 1, 10, 100]
param: binarize , options: [0, 0.25, 0.5, 0.75, 1]
param: fit_prior , options: ['true', 'false']
ml: GaussianNB
ml: MultinomialNB
param: alpha , options: [0.001, 0.01, 0.1, 1.0, 10, 100]
param: fit_prior , options: ['true', 'false']
ml: DecisionTreeClassifier
param: criterion , options: ['gini', 'entropy']
param: max_depth , options: [2, 4, 6, 8, 'None']
param: min_samples_split , options: [2, 5, 10, 20]
param: min_samples_leaf , options: [1, 5, 10, 20]
param: min_weight_fraction_leaf , options: [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
param: max_features , options: ['sqrt', 'log2', 'None']
ml: ExtraTreesClassifier
param: n_estimators , options: [100, 250]
param: criterion , options: ['gini', 'entropy']
param: max_features , options: ['sqrt', 'log2', 'None']
param: min_samples_split , options: [2, 5, 10, 20]
param: min_samples_leaf , options: [1, 5, 10, 20]
param: bootstrap , options: ['true', 'f

## method to extend ML parameter options with default parameters if not present

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
default_params = {}
default_params['DecisionTreeClassifier'] = DecisionTreeClassifier().get_params()
default_params['RandomForestClassifier'] = RandomForestClassifier().get_params()
default_params['ExtraTreesClassifier'] = ExtraTreesClassifier().get_params()
default_params['GradientBoostingClassifier'] = GradientBoostingClassifier().get_params()
default_params['BernoulliNB'] = BernoulliNB().get_params()
default_params['GaussianNB'] = GaussianNB().get_params()
default_params['MultinomialNB'] = MultinomialNB().get_params()
default_params['KNeighborsClassifier'] = KNeighborsClassifier().get_params()
default_params['LinearSVC'] = LinearSVC().get_params()
default_params['SVC'] = SVC().get_params()
default_params['LogisticRegression'] = LogisticRegression().get_params()

def extend_params(ml,params):
    missing_params = [(k,v) for k,v in default_params[ml].items() if k not in params]
    for k,v in missing_params:
        params[k] = v
    return params

##
# function to lower case strings
def fix_params(params):
    for k,v in params.items():
        if type(v) is str:
            if v == 'true':
                params[k] = True
            elif v == 'false':
                params[k] = False
            elif v == 'None':
                params[k] == None
            else:
                params[k] = v.lower() 
    return params

In [3]:
pennai_ml_opts_ext = {}
for k,v in pennai_ml_opts.items():
    pennai_ml_opts_ext[k] = []
    for params in v:
        pennai_ml_opts_ext[k].append(extend_params(k,params))

# load pmlb results
filter out classifiers not in PennAI

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('../../mock_experiment/sklearn-benchmark5-data-edited.tsv.gz', sep='\t', names=['dataset',
                                                                     'classifier',
                                                                     'parameters',
                                                                     'accuracy', 
                                                                     'macrof1',
                                                                     'bal_accuracy']).fillna('')
print(data.head())
data['accuracy'] = data['accuracy'].apply(lambda x: round(x, 3))
print('loaded ',data['dataset'].unique().shape[0],'datasets and ', data['classifier'].unique().shape[0],'classifiers')
# subset data to classifiers used in PennAI
pennai_classifiers = ['LogisticRegression', 'RandomForestClassifier', 'SVC', 
                      'KNeighborsClassifier', 'DecisionTreeClassifier', 'GradientBoostingClassifier']
mask = np.array([c in pennai_classifiers for c in data['classifier'].values])
data = data.loc[mask,:]
print('datasets (',len(data['dataset'].unique()),')')
print('classifiers (',len(data['classifier'].unique()),'):',data['classifier'].unique())
for ml, df_g in data.groupby('classifier'):
    print('parameters for ',ml,'(',len(df_g['parameters'].unique()),'):',df_g['parameters'].unique()[:5])


/home/bill/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


                                             dataset          classifier  \
0  GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...  AdaBoostClassifier   
1  GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...  AdaBoostClassifier   
2  GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...  AdaBoostClassifier   
3  GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...  AdaBoostClassifier   
4  GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...  AdaBoostClassifier   

                             parameters  accuracy   macrof1 bal_accuracy  
0    learning_rate=0.01,n_estimators=10  0.479375  0.470857   0.479375.1  
1   learning_rate=0.01,n_estimators=100  0.476875  0.476497     0.476875  
2  learning_rate=0.01,n_estimators=1000  0.488125  0.488115     0.488125  
3    learning_rate=0.01,n_estimators=50  0.483750  0.483652      0.48375  
4   learning_rate=0.01,n_estimators=500  0.495625  0.495609     0.495625  
loaded  166 datasets and  13 classifiers
datasets ( 166 )
classifiers ( 6 ): ['DecisionTreeCl

In [2]:
data.loc[data.classifier=='GradientBoostingClassifier']['parameters'].unique()

array(['loss=deviance,learning_rate=0.01,n_estimators=10,max_depth=1,max_features=0.2',
       'loss=deviance,learning_rate=0.01,n_estimators=10,max_depth=1,max_features=0.3',
       'loss=deviance,learning_rate=0.01,n_estimators=10,max_depth=1,max_features=0.4',
       ...,
       'loss=exponential,learning_rate=50.0,n_estimators=500,max_depth=None,max_features=0.9',
       'loss=exponential,learning_rate=50.0,n_estimators=500,max_depth=None,max_features=log2',
       'loss=exponential,learning_rate=50.0,n_estimators=500,max_depth=None,max_features=sqrt'],
      dtype=object)

# convert from sklearn-style parameter formatting to dictionary

In [5]:
def fix_params(p):
    fixed_params = {k:v for k,v in [tuple(ps.split('=')) for ps in filter(None, p.split(','))]}
    for k,v in fixed_params.items():
        try:
            fixed_params[k] = int(v)
        except ValueError:
            try:     
                fixed_params[k] = float(v)
            except ValueError:
                if fixed_params[k] == 'None':
                    fixed_params[k] = None 
                else:
                    fixed_params[k] = str(v).lower()
                pass
    return fixed_params
# datanew = data.copy()
data_formatted = data.copy()
data_formatted['parameters'] = data_formatted['parameters'].apply(lambda x: fix_params(x))

data_formatted.head()

,dataset,classifier,parameters,accuracy,macrof1,bal_accuracy
175,GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...,DecisionTreeClassifier,"{'min_weight_fraction_leaf': 0.0, 'max_feature...",0.507,0.507488,0.5075
176,GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...,DecisionTreeClassifier,"{'min_weight_fraction_leaf': 0.0, 'max_feature...",0.476,0.476040,0.47625
177,GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...,DecisionTreeClassifier,"{'min_weight_fraction_leaf': 0.0, 'max_feature...",0.507,0.506832,0.506875
178,GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...,DecisionTreeClassifier,"{'min_weight_fraction_leaf': 0.0, 'max_feature...",0.490,0.489993,0.49
179,GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...,DecisionTreeClassifier,"{'min_weight_fraction_leaf': 0.0, 'max_feature...",0.512,0.512497,0.5125


## add default parameters to parameter dictionaries

In [6]:
def df_extend_params(x):
#     print(x)
    x.parameters = extend_params(x.classifier,x.parameters)
    return x
data_formatted_ext = data_formatted.copy()
data_formatted_ext = data_formatted_ext.apply(lambda x: df_extend_params(x),axis=1)

In [7]:
data_formatted_ext.head()

,dataset,classifier,parameters,accuracy,macrof1,bal_accuracy
175,GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...,DecisionTreeClassifier,"{'min_weight_fraction_leaf': 0.0, 'max_feature...",0.507,0.507488,0.5075
176,GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...,DecisionTreeClassifier,"{'min_weight_fraction_leaf': 0.0, 'max_feature...",0.476,0.476040,0.47625
177,GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...,DecisionTreeClassifier,"{'min_weight_fraction_leaf': 0.0, 'max_feature...",0.507,0.506832,0.506875
178,GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...,DecisionTreeClassifier,"{'min_weight_fraction_leaf': 0.0, 'max_feature...",0.490,0.489993,0.49
179,GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...,DecisionTreeClassifier,"{'min_weight_fraction_leaf': 0.0, 'max_feature...",0.512,0.512497,0.5125


# filter out any parameter combinations that PennAI can't recommend

In [8]:
def valid_param_combo(ml, params):
    if params in pennai_ml_opts_ext[ml]:
        return True
    else:
        return False
print('unfiltered:',len(data_formatted_ext),'results with',
     len(data_formatted_ext.parameters.apply(str).unique()),'unique parameters')
data_formatted_ext_filtered = data_formatted_ext#.loc[data_formatted_ext.classifier=='SVC']
print('calculating mask...')
mask = []
from tqdm import tqdm
for _, row in tqdm(data_formatted_ext_filtered.iterrows()):
    mask.append(valid_param_combo(row['classifier'],row['parameters']) )
print('done with mask')
# mask = mask | data_formatted_ext_filtered.classifier=='LogisticRegression'
data_formatted_ext_filtered = data_formatted_ext_filtered.loc[mask]
print('filtered data has',len(data_formatted_ext_filtered),'results with',
     len(data_formatted_ext_filtered.parameters.apply(str).unique()),'unique parameters')
data_formatted_ext_filtered.head()
# data_filtered = data.loc[lambda x: valid_param_combo(i['classifier'],i['parameters']) for i in x]


0it [00:00, ?it/s]

unfiltered: 1113067 results with 8758 unique parameters
calculating mask...


1113067it [03:49, 4860.09it/s]


done with mask
filtered data has 80632 results with 623 unique parameters


,dataset,classifier,parameters,accuracy,macrof1,bal_accuracy
185,GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...,DecisionTreeClassifier,"{'min_weight_fraction_leaf': 0.0, 'max_feature...",0.479,0.479267,0.479375
186,GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...,DecisionTreeClassifier,"{'min_weight_fraction_leaf': 0.0, 'max_feature...",0.502,0.502493,0.5025
187,GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...,DecisionTreeClassifier,"{'min_weight_fraction_leaf': 0.0, 'max_feature...",0.498,0.498109,0.498125
188,GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...,DecisionTreeClassifier,"{'min_weight_fraction_leaf': 0.0, 'max_feature...",0.503,0.503109,0.503125
199,GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...,DecisionTreeClassifier,"{'min_weight_fraction_leaf': 0.05, 'max_featur...",0.516,0.515156,0.51625


In [9]:
for clf, dfg in data_formatted_ext_filtered.groupby('classifier'):
    print(clf,'unique params combos:',len(dfg.parameters.apply(str).unique()))
alg_params = data_formatted_ext_filtered['classifier']+'|'+data_formatted_ext_filtered['parameters'].apply(str)
print(len(alg_params.unique()),'unique algorithm/parameter combinations')

DecisionTreeClassifier unique params combos: 32
GradientBoostingClassifier unique params combos: 36
KNeighborsClassifier unique params combos: 12
LogisticRegression unique params combos: 18
SVC unique params combos: 525
623 unique algorithm/parameter combinations


# re-prune parameters from knowledgebase that were added as defaults (not in PennAI)

In [10]:
def prune_params(x):
#     pdb.set_trace()
    ml = x.classifier
    params = x.parameters
    param_opts = original_pennai_ml_opts[ml]
    x.parameters = {k:v for k,v in params.items() if k in param_opts}
    return x 
data_formatted_ext_filtered_pruned = data_formatted_ext_filtered.copy()
data_formatted_ext_filtered_pruned = data_formatted_ext_filtered_pruned.apply(
    lambda x: prune_params(x),axis=1)
    


In [11]:
# turn parameters into sorted lists

In [12]:
for clf, dfg in data_formatted_ext_filtered_pruned.groupby('classifier'):
    print(clf,'unique params combos:',len(dfg.parameters.apply(str).unique()))
alg_params = (data_formatted_ext_filtered_pruned['classifier']+'|'
              +data_formatted_ext_filtered_pruned['parameters'].apply(str))
print(len(alg_params.unique()),'unique algorithm/parameter combinations')

DecisionTreeClassifier unique params combos: 32
GradientBoostingClassifier unique params combos: 36
KNeighborsClassifier unique params combos: 12
LogisticRegression unique params combos: 18
SVC unique params combos: 525
623 unique algorithm/parameter combinations


In [13]:
# Remove big files since their metafeatures take forever to load
data_formatted_ext_filtered_pruned_small = data_formatted_ext_filtered_pruned.copy()
#datasets to remove:
big_datasets = ['poker', 'kddcup', 'sleep', 'fars', 'mnist'] 
# , 'connect-4', 'shuttle', 'adult', 'krkopt', 
#                 'letter', 'magic', 'nursery', 'pendigits', 'coil2000', 'agaricus-lepiota','optdigits']
mask = np.array([d not in big_datasets for d in data_formatted_ext_filtered_pruned_small['dataset'].values])
data_formatted_ext_filtered_pruned_small  = data_formatted_ext_filtered_pruned_small.loc[mask,:]
print(len(data_formatted_ext_filtered_pruned_small['dataset'].unique()),'datasets left')

161 datasets left


# write modified data to file

In [16]:
print(len(data_formatted_ext_filtered_pruned_small),'total results')
data_formatted_ext_filtered_pruned_small.rename(columns={'classifier':'algorithm'},inplace=True)
data_formatted_ext_filtered_pruned_small.to_csv('sklearn-benchmark5-data-knowledgebase.tsv.gz',
                                                compression='gzip',index=False,sep='\t')

80302 total results


In [17]:
# make a small knowledge base for testing
test_data = data_formatted_ext_filtered_pruned_small.sample(frac=0.1)
data_formatted_ext_filtered_pruned_small.to_csv('sklearn-benchmark5-data-knowledgebase-small.tsv.gz',
                                                compression='gzip',index=False,sep='\t')